In [1]:
import pandas as pd

In [2]:
test_excel_path = "pcast_test.xlsx"
test_df = pd.read_excel(test_excel_path)
test_df.head()

,Asset ID,Quartex UniqueIdentifier,Quartex Name,Collection(s),Published URL,Title,People and Organizations,Location,Approximate Date,Abstract,...,University Archives,Time Span,Date Digital,Digitization Specifications,Original Handle,Rights Summary,Accessibility Feature,Accessibility Summary,Legacy Subjects,Related Agencies and Organizations
0,266404,6ccc3880-63b2-4b06-ae3f-e86ff875452d,pcast0001,PCAST reports|Barack Obama Science Policy Coll...,digitalcollections.rice.edu/documents/detail/r...,Report to the President on U.S. Preparations f...,"Lander, Eric S.|Varmus, Harold|President's Cou...",NaN,NaN,NaN,...,NaN,2000s,2021.0,This asset is born digital. It may not have a ...,https://hdl.handle.net/1911/112990,Public domain,taggedPDF,NaN,United States. President (2009-2017 : Obama)|H...,NaN
1,266405,61af38e3-1692-4121-b245-c52d2848f67b,pcast0005,PCAST reports|Barack Obama Science Policy Coll...,digitalcollections.rice.edu/documents/detail/r...,Report to the President Realizing the Full Pot...,"Mundie, Craig|Cassel, Christine K. (Christine ...",NaN,NaN,NaN,...,NaN,2010s,2021.0,This asset is born digital. It may not have a ...,https://hdl.handle.net/1911/112994,Public domain,taggedPDF,NaN,United States. President (2009-2017 : Obama)|I...,NaN
2,266406,1227dae8-2cb7-487a-9815-de50492a1f9a,pcast0003,PCAST reports|Barack Obama Science Policy Coll...,digitalcollections.rice.edu/documents/detail/r...,Report to the President and Congress Designing...,"Lazowska, Edward D.|Shaw, David E.|President's...",NaN,NaN,NaN,...,NaN,2010s,2021.0,This asset is born digital. It may not have a ...,https://hdl.handle.net/1911/112992,Public domain,taggedPDF,NaN,United States. President (2009-2017 : Obama)|C...,NaN
3,266407,4c4edf2b-8dd9-4cd3-8148-ec7b0b8b842e,pcast0004,PCAST reports|Barack Obama Science Policy Coll...,digitalcollections.rice.edu/documents/detail/r...,Report to the President Prepare and Inspire: K...,"Lander, Eric S.|Gates, S. James, Jr., 1950-|Pr...",NaN,NaN,NaN,...,NaN,2010s,2021.0,This asset is born digital. It may not have a ...,https://hdl.handle.net/1911/112993,Public domain,taggedPDF,NaN,United States. President (2009-2017 : Obama)|S...,NaN
4,266408,0530c901-140c-46c1-82b2-e68c3e96d37c,pcast0002,PCAST reports|Barack Obama Science Policy Coll...,digitalcollections.rice.edu/documents/detail/r...,Report to the President on Accelerating the Pa...,"Savitz, Maxine|Moniz, Ernest J.|President's Co...",NaN,NaN,NaN,...,NaN,2010s,2021.0,This asset is born digital. It may not have a ...,https://hdl.handle.net/1911/112991,Public domain,taggedPDF,NaN,United States. President (2009-2017 : Obama)|E...,NaN


In [3]:
test_df["Date"]

0       20090807
1         201010
2         201012
3         201009
4         201011
          ...   
1517      202206
1518      202107
1519      202206
1520    20001116
1521    20000728
Name: Date, Length: 1522, dtype: int64

In [52]:
# Mundie, Craig|Cassel, Christine K. (Christine Karen), 1945-|President's Council of Advisors on Science and Technology (PCAST)
exp_lst = test_df["People and Organizations"][68].split("|")
cleaned_lst = []
for exp in exp_lst:
    exp = exp.strip()
    if "(PCAST)" in exp:
        cleaned_lst.append(exp[:-7])
    elif "," in exp:
        if  "(" in exp and ")" in exp:
            lparen = exp.index("(") + 1
            rparen = exp.index(")")
            comma_idx = exp.index(",")
            cleaned_lst.append(exp[lparen:rparen] + " " + exp[:comma_idx])
        else:
            comma_idx = exp.index(",")
            if "," in exp[comma_idx+1:]:
                comma_idx_2 = exp[comma_idx+1:].index(",")
                cleaned_lst.append(exp[comma_idx+1: comma_idx + comma_idx_2+1]+ " " + exp[:comma_idx])
            else:
                cleaned_lst.append(exp[comma_idx+1: ]+ " " + exp[:comma_idx])
    else:
        cleaned_lst.append(exp)

In [53]:
cleaned_lst

[' Peter H. Raven',
 ' Alfonso Alonso',
 ' Elise F. Granek',
 ' Francisco Dallmeier',
 "President's Council of Advisors on Science and Technology "]

In [6]:
test_df["Description"]

0       The novel influenza A (H1N1) virus (2009-H1N1)...
1       New information technologies (IT) in healthcar...
2       The Federal Networking and Information Technol...
3       U.S. leadership in science, technology, engine...
4       Energy security and the transition to affordab...
                              ...                        
1517    Oral history interview with Barbara Schaal (tw...
1518    Oral history interview with Luis M. Proenza (t...
1519    Published transcript of oral history interview...
1520    Document outlining talking points for a teleph...
1521    Notes by Neal Lane, Director of the Office of ...
Name: Description, Length: 1522, dtype: object

In [7]:
test_df["Subject"]

0             Politics and government|Health and medicine
1             Politics and government|Health and medicine
2                                 Politics and government
3                                 Politics and government
4                                 Politics and government
                              ...                        
1517    Technology|Academics|Emigration and immigratio...
1518    Academics|Economics and finance|Technology|Pol...
1519    Technology|Academics|Emigration and immigratio...
1520                              Politics and government
1521                              Politics and government
Name: Subject, Length: 1522, dtype: object

In [69]:
benchmark_excel_path = "/Users/amarkanaka/repos/Baker_Project/gemini_benchmark.csv"
benchmark_df = pd.read_csv(benchmark_excel_path)
benchmark_df.head()

,Unique_ID,Collection(s),Published URL,Title,People and Organizations,Description,Language,Publisher,Date,Source,Subject,Format Genre,Accessibility Summary
0,pcast0001,Null,www.ostp.gov/cs/pcast,REPORT TO THE PRESIDENT ON U.S. PREPARATIONS F...,"['Executive Office of the President', ""Preside...",Report to the President on U.S. Preparations f...,en,"Executive Office of the President, President's...",7-Aug-09,Null,"2009-H1N1 Influenza, U.S. Preparations, Scienc...",Report,Null
1,pcast0005,Null,http://www.whitehouse.gov/ostp/pcast,REPORT TO THE PRESIDENT REALIZING THE FULL POT...,President's Council of Advisors on Science and...,A report by the President's Council of Advisor...,en,"Executive Office of the President, President's...",Dec-10,Null,"Health Information Technology, Healthcare, Sci...",Report,Null
2,pcast0003,Null,http://www.whitehouse.gov/ostp/pcast,DESIGNING A DIGITAL FUTURE: FEDERALLY FUNDED R...,"['Executive Office of the President', ""Preside...",This report assesses the status and direction ...,en,"Executive Office of the President, President's...",Dec-10,Null,Networking and Information Technology Research...,report,Null
3,pcast0004,Null,http://www.whitehouse.gov/ostp/pcast,REPORT TO THE PRESIDENT PREPARE AND INSPIRE: K...,"['Executive Office of the President', ""Preside...",This report provides a strategy for improving ...,en,President's Council of Advisors on Science and...,Sep-10,Null,"K-12 Education in Science, Technology, Enginee...",Report,Null
4,pcast0002,Null,http://www.whitehouse.gov/ostp/pcast,REPORT TO THE PRESIDENT ON ACCELERATING THE PA...,"John P. Holdren, Eric Lander, Rosina Bierbaum,...",This report addresses how to transform the ene...,en,"Executive Office of the President, President's...",Nov-10,Null,"Energy Technologies, Federal Energy Policy, En...",Report,Null


In [70]:
benchmark_df = benchmark_df.rename(columns={"Unique_ID":"Quartex Name"})

In [10]:
benchmark_df[" People and Organizations"][0][1:-1].split(",")

["'Executive Office of the President'",
 ' "President\'s Council of Advisors on Science and Technology"',
 " 'John P. Holdren'",
 " 'Office of Science and Technology Policy'",
 " 'Eric Lander'",
 " 'Broad Institute of Harvard and MIT'",
 " 'Harold Varmus'",
 " 'Memorial Sloan-Kettering Cancer Center'",
 " 'Rosina Bierbaum'",
 " 'University of Michigan'",
 " 'Christine Cassel'",
 " 'American Board of Internal Medicine'",
 " 'Christopher Chyba'",
 " 'Princeton University'",
 " 'S. James Gates",
 " Jr.'",
 " 'University of Maryland",
 " College Park'",
 " 'Shirley Ann Jackson'",
 " 'Rensselaer Polytechnic Institute'",
 " 'Richard C. Levin'",
 " 'Yale University'",
 " 'Chad Mirkin'",
 " 'Northwestern University'",
 " 'Mario Molina'",
 " 'University of California",
 " San Diego'",
 " 'Scripps Institution of Oceanography'",
 " 'Ernest J. Moniz'",
 " 'Massachusetts Institute of Technology'",
 " 'Craig Mundie'",
 " 'Microsoft Corporation'",
 " 'Ed Penhoet'",
 " 'Alta Partners'",
 " 'Immune Des

In [76]:
from fuzzywuzzy import fuzz
# people and organizations benchmark
def ppl_orgs_recall_match_v1(gt_entry, bm_list, recall_threshold=0.75, fuzzy_threshold=60):
    missed_count_short = 0
    missed_count_long = 0
    # get list of benchmark names
    bm_clean = bm_list
    if bm_clean[0] == "[" :
        bm_clean = bm_clean[1:]
    if bm_clean[-1] == "]":
        bm_clean = bm_clean[:-1]
    bm_clean = bm_clean.split(",")

    """
    get list of ground truth names and orgs, taking out dates, flipping names, and
    removing parentheses at the same time
    """

    gt_entry_lst = gt_entry.split("|")
    cleaned_lst = []
    for exp in gt_entry_lst:
        exp = exp.strip()
        if "(PCAST)" in exp:
            cleaned_lst.append(exp[:-7])
        elif "," in exp:
            if  "(" in exp and ")" in exp:
                lparen = exp.index("(") + 1
                rparen = exp.index(")")
                comma_idx = exp.index(",")
                cleaned_lst.append(exp[lparen:rparen] + " " + exp[:comma_idx])
            else:
                comma_idx = exp.index(",")
                if "," in exp[comma_idx+1:]:
                   comma_idx_2 = exp[comma_idx+1:].index(",")
                   cleaned_lst.append(exp[comma_idx+1: comma_idx + comma_idx_2+1]+ " " + exp[:comma_idx])
                else:
                    cleaned_lst.append(exp[comma_idx+1: ]+ " " + exp[:comma_idx])
        else:
            cleaned_lst.append(exp)
    
    match_count = 0

    for gt_entity in cleaned_lst:
        # Check if any benchmark entity matches this ground truth entity
        for bm_entity in bm_clean:
            if fuzz.token_set_ratio(gt_entity, bm_entity) >= fuzzy_threshold:
                # print(bm_entity)
                # print(gt_entity)
                match_count += 1
                break

    # percentage of ground truth found
    recall = match_count / len(cleaned_lst)
   
    if recall < recall_threshold:
        if len(cleaned_lst) > 3:
            missed_count_long += 1
            # print(cleaned_lst)
            # print(bm_clean)
            # print("\n")
        else:
            missed_count_short += 1

        
    return 1 if recall >= recall_threshold else 0, missed_count_long, missed_count_short

num = ppl_orgs_recall_match_v1(test_df["People and Organizations"][1], benchmark_df[" People and Organizations"][1])
num

(1, 0, 0)

Benchmarking People and Organizations

In [14]:
ppl_orgs_df = benchmark_df.merge(test_df, how='inner', on='Quartex Name')

In [72]:
gt = test_df["People and Organizations"]
quartex_names = set(benchmark_df["Quartex Name"].to_numpy())
len(quartex_names)

781

In [73]:
quartex_id = test_df["Quartex Name"].to_numpy()

In [74]:
gt = test_df["People and Organizations"].to_numpy()
gt_clean = [gt[idx] for idx in range(len(gt)) if quartex_id[idx] in quartex_names]

In [ ]:
gt_clean

In [ ]:
bm

In [77]:
bm = benchmark_df[" People and Organizations"].to_numpy()
matches = 0
missed_count_long = 0
missed_count_short = 0
for gt_entry, bm_entry in zip(gt_clean, bm):
    match, missed_count_l, missed_count_s = ppl_orgs_recall_match_v1(gt_entry, bm_entry)
    matches += match
    missed_count_long += missed_count_l
    missed_count_short += missed_count_s
print("matches: ")
print(matches/len(gt_clean))
print("\n")
print("missed count for longer entries: ")
print(missed_count_long)
print("\n")
print("missed count for shorter entries: ")
print(missed_count_short)

matches: 
0.6197183098591549


missed count for longer entries: 
22


missed count for shorter entries: 
275


Description Accuracy

In [20]:
string1 = "This is a sentence"
set(string1.split(" "))

{'This', 'a', 'is', 'sentence'}

In [108]:
def WER_BoW(gt, bm):
    gt_set = set([word.strip().lower() for word in gt.split(" ")])
    bm_set = set([word.strip().lower() for word in bm.split(" ")])
    
    # print(bm_set)
    # print(gt_set)

    gt_N = len(gt_set)
    bm_gt_CARD = len(gt_set.intersection(bm_set))

    wer_bow = (gt_N - bm_gt_CARD) / gt_N

    # print(wer_bow)

    return wer_bow



In [87]:
for entry in gt_clean:
    if isinstance(entry, float):
        print(entry)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
benchmark_excel_path_2 = "/Users/amarkanaka/repos/Baker_Project/mistral_gemini_benchmark.csv"
benchmark_df_2 = pd.read_csv(benchmark_excel_path_2)
benchmark_df_2 = benchmark_df_2.rename(columns={"Unique_ID":"Quartex Name"})

bm_description = benchmark_df_2[" Description"].to_numpy()
quartex_names = set(benchmark_df_2["Quartex Name"].to_numpy())

gt = test_df["Description"].to_numpy()
gt_clean = [gt[idx] for idx in range(len(gt)) if quartex_id[idx] in quartex_names]

wer_bow_avg = 0
for bm_desc, gt_desc in zip(bm_description, gt_clean):
    if isinstance(gt_desc, float):
        continue
    wer_bow_avg += WER_BoW(gt_desc, bm_desc)
    if wer_bow_avg > 0.8:
        print("Ground truth: ")
        print(gt_desc)
        print("Benchmark: ")
        print(bm_desc)
        print("-----------------------")
print(wer_bow_avg / len(bm_description))

Title Accuracy

In [126]:
import numpy as np
benchmark_excel_path_2 = "/Users/amarkanaka/repos/Baker_Project/mistral_gemini_benchmark.csv"
benchmark_df_2 = pd.read_csv(benchmark_excel_path_2)
benchmark_df_2 = benchmark_df_2.rename(columns={"Unique_ID":"Quartex Name"})

bm_description = benchmark_df_2[" Title"].to_numpy()
quartex_names = set(benchmark_df_2["Quartex Name"].to_numpy())

gt = test_df["Title"].to_numpy()
gt_clean = [gt[idx] for idx in range(len(gt)) if quartex_id[idx] in quartex_names]

wer_bow_avg = []
for bm_desc, gt_desc in zip(bm_description, gt_clean):
    if isinstance(gt_desc, float):
        continue
    score = WER_BoW(gt_desc, bm_desc)
    wer_bow_avg.append(score)
    if score > 0.8:
        print("Ground truth: ")
        print(gt_desc)
        print("Benchmark: ")
        print(bm_desc)
        print("-----------------------")

print("mean: ")
print(np.mean(wer_bow_avg))

print("stdev: ")
print(np.std(wer_bow_avg))

print("median: ")
print(np.median(wer_bow_avg, axis = 0))

Ground truth: 
Aging America & Hearing Loss: Imperative of Improved Hearing Technologies
Benchmark: 
EXECUTIVE OFFICE OF THE PRESIDENT PRESIDENT'S COUNCIL OF ADVISORS ON SCIENCE AND TECHNOLOGY
-----------------------
Ground truth: 
Action Needed to Protect Against Biological Attack
Benchmark: 
EXECUTIVE OFFICE OF THE PRESIDENT  PRESIDENT'S COUNCIL OF ADVISORS ON SCIENCE AND TECHNOLOGY
-----------------------
Ground truth: 
Information Technology for Targeting Job-Skills Training and Matching Talent to Jobs
Benchmark: 
Advancing Technology-Enabled Training for the Middle-Skill Labor Market
-----------------------
Ground truth: 
Report to the President Revitalizing the U.S. Semiconductor Ecosystem
Benchmark: 
POWERFUL PARTNERSHIPS: THE FEDERAL ROLE IN INTERNATIONAL COOPERATION ON ENERGY INNOVATION
-----------------------
Ground truth: 
Powerful Partnerships: The Federal Role in International Cooperation on Energy Innovation
Benchmark: 
Wellspring of Prosperity: Science and Technology in 

In [124]:
num = 0
for score in wer_bow_avg:
    if score <=0.3:
        num+=1
print(num/len(wer_bow_avg))
print(num)

0.054282267792521106
45


Date Accuracy

In [127]:
import pandas as pd
from dateutil.parser import parse

bm_date = benchmark_df[" Date"].to_numpy()
quartex_names = set(benchmark_df["Quartex Name"].to_numpy())
gt = test_df["Date"].to_numpy()
gt_clean_date = [gt[idx] for idx in range(len(gt)) if quartex_id[idx] in quartex_names]


def normalize_date(date_str):
    try:
        return parse(date_str, fuzzy=True).date().isoformat()
    except:
        return date_str.strip().lower()

count = 0
for bm_entry, gt_entry in zip(bm_date, gt_clean_date):
    if normalize_date(gt_entry) == normalize_date(bm_entry):
        count += 1
    else:
        print(normalize_date(gt_entry))
        print(normalize_date(bm_entry))

print(count/len(bm_date))


AttributeError: 'numpy.int64' object has no attribute 'strip'